In [1]:
import numpy as np

from Modeling import soeModel
%load_ext autoreload
%autoreload 1
%reload_ext autoreload
import pandas as pd
import re
import json

In [2]:
model = soeModel()
model.preprocessing()

In [3]:
with open("local_dictionary.json", "r") as f:
    translations_all = json.load(f)

In [4]:
%%time
model.translate_cells(local_dict_name="local_dictionary.json", local=True)

CPU times: user 22.9 ms, sys: 1.22 ms, total: 24.1 ms
Wall time: 23.3 ms


In [20]:
# model.power_sale

In [6]:
model.correct_digital_type()

In [7]:
model.power_sale

,Power Plant Classification,power plant,Power grid,geographic location,Total power generation (100 million kWh) 2021,Total electricity sales (100 million kWh) 2021,Average utilization hours (hours) 2021,Standard coal consumption for electricity sales (g/kWh) 2021,Electricity sales price (yuan/MWh) 2021,Total installed capacity by the end of the year (MW) 2021,...,Electricity Sales Price (RMB/MWh) 2019,Total installed capacity at the end of the year (MW) 2019,Equity installed capacity at the end of the year (MW) 2019,Total power generation (100 million kWh) 2018,Total electricity sales (100 million kWh) 2018,Average utilization hours (hours) 2018,Standard coal consumption for electricity sales (g/kWh) 2018,Electricity sales price (yuan/MWh) 2018,Total installed capacity at the end of the year (MW) 2018,Equity installed capacity at the end of the year (MW) 2018
0,coal burning,Zhuneng Power,North China Power Grid,Inner Mongolia,36.3,32.9,5504.0,347.0,299.0,660.0,...,230.0,660.0,381.0,33.0,29.0,3392.0,367.0,213.0,960.0,554.0
1,coal burning,Guohua Jungar,North China Power Grid,Inner Mongolia,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,71.0,65.0,5411.0,310.0,236.0,1320.0,639.0
2,coal burning,Guohua calls,North China Power Grid,Inner Mongolia,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,60.0,54.0,5019.0,321.0,236.0,1200.0,960.0
3,coal burning,Shendong Power,Northwest/North China/\nShaanxi Province Local...,Inner Mongolia,240.3,222.1,4792.0,320.0,323.0,5014.0,...,294.0,5814.0,5328.0,335.0,307.0,3723.0,337.0,255.0,8984.0,8726.0
4,coal burning,Victory Energy,North China Power Grid,Inner Mongolia,9.4,8.4,712.0,352.0,278.0,1320.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,coal burning,Cangdong Power,North China Power Grid,Hebei,114.4,108.8,4539.0,297.0,323.0,2520.0,...,318.0,2520.0,1285.0,132.0,126.0,5244.0,300.0,315.0,2520.0,1285.0
6,coal burning,Sanhe Power,North China Power Grid,Hebei,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,68.0,64.0,5119.0,291.0,327.0,1330.0,512.0
7,coal burning,Dingzhou Electric Power,North China Power Grid,Hebei,117.6,108.8,4666.0,304.0,324.0,2520.0,...,315.0,2520.0,1021.0,139.0,129.0,5522.0,307.0,320.0,2520.0,1021.0
8,coal burning,Taishan Electric Power,China Southern Power Grid,Guangdong,267.1,252.4,5218.0,307.0,394.0,5120.0,...,391.0,5090.0,4072.0,239.0,226.0,4694.0,309.0,354.0,5090.0,4072.0
9,coal burning,Huizhou Thermal Power,China Southern Power Grid,Guangdong,38.0,34.3,5760.0,307.0,371.0,660.0,...,377.0,660.0,660.0,34.0,31.0,5187.0,324.0,358.0,660.0,660.0


In [8]:
params = {
    "types": "power",
    "pct": 0.5,
    "year": 2019,
    "coal_price": 50.0,
    "coal_power": True,
    "coal_purchase": False,
    "coal_transportation": False
}
table = model.scenario(**params)

/Users/zechengli/Desktop/CHINA_SOE_MODEL/Modeling.py:234: RuntimeWarning: invalid value encountered in divide
  lst2 = (np.array(lst1) / np.array(sale_cost_lst)).tolist()


In [9]:
table

,Power Dispatched (TWh),Weighted Average Tariff (RMB/MWh),Estimated Gross Revenue (RMB/MWh),Estimated Sales Cost (Million RMB),Gross Profit (Million RMB),Gross Profitability
All Power,130.33,50.0,44012.16,NaN,NaN,nan%
Coal Power,126.52,50.0,41829.03,33560.0,8269.03,24.64%
Gas Power,3.81,50.0,2183.13,NaN,NaN,nan%
Hydro Power,0.00,50.0,0.00,NaN,NaN,nan%


In [10]:
model.case

,Reduced by 50%
Power Dispatched (TWh),63.26
Weighted Average Tariff (RMB/MWh),50.0
Estimated Gross Revenue (RMB/MWh),3163.0
Estimated Sales Cost (Million RMB),22145.468
Gross Profit (Million RMB),-18982.468
Gross Profitability,-85.72%


In [11]:
#TODO: change the table structure

In [12]:
model = soeModel()
model.df_lst

['power_sale', 'power_cost', 'elec_cost']

In [13]:
model.power_sale

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,NaN,NaN,NaN,NaN,2021,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN
1,电厂分类,电厂,所在电网,地理位置,总发电量\n(亿千瓦时),总售电量\n(亿千瓦时),平均利用小时\n(小时),售电标准煤耗\n(克/千瓦时),售电电价\n(元/兆瓦时),年底总装机容量\n(兆瓦),...,售电电价\n(元/兆瓦时),年底总装机容量\n(兆瓦),年底权益装机容量\n(兆瓦),总发电量\n(亿千瓦时),总售电量\n(亿千瓦时),平均利用小时\n(小时),售电标准煤耗\n(克/千瓦时),售电电价\n(元/兆瓦时),年底总装机容量\n(兆瓦),年底权益装机容量\n(兆瓦)
2,燃煤,准能电力,华北电网,内蒙古,36.3,32.9,"5,504.0",347.0,299.0,660.0,...,230.0,660,381,33,29,"3,392",367,213,960,554
3,燃煤,国华准格尔,华北电网,内蒙古,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,71,65,"5,411",310,236,"1,320",639
4,燃煤,国华呼电,华北电网,内蒙古,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,60,54,"5,019",321,236,"1,200",960
5,燃煤,神东电力,西北/华北/\n陕西省地方电网,内蒙古,240.3,222.1,"4,792.0",320.0,323.0,"5,014.0",...,294.0,"5,814","5,328",335,307,"3,723",337,255,"8,984","8,726"
6,燃煤,胜利能源,华北电网,内蒙古,9.4,8.4,712.0,352.0,278.0,"1,320.0",...,-,-,-,NaN,-,NaN,NaN,-,-,-
7,燃煤,沧东电力,华北电网,河北,114.4,108.8,"4,539.0",297.0,323.0,"2,520.0",...,318.0,"2,520","1,285",132,126,"5,244",300,315,"2,520","1,285"
8,燃煤,三河电力,华北电网,河北,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,68,64,"5,119",291,327,"1,330",512
9,燃煤,定州电力,华北电网,河北,117.6,108.8,"4,666.0",304.0,324.0,"2,520.0",...,315.0,"2,520","1,021",139,129,"5,522",307,320,"2,520","1,021"


In [14]:
model.power_cost

,0,1,2,3,4,5,6,7,8,9
0,NaN,2021年,NaN,NaN,2020年,NaN,NaN,2019年,NaN,NaN
1,NaN,成本\n(百万元),售电量\n(亿千瓦时),单位成本\n(元/兆瓦时),成本\n(百万元),售电量\n(亿千瓦时),单位成本\n(元/兆瓦时),成本\n(百万元),售电量\n(亿千瓦时),单位成本\n(元/兆瓦时)
2,售电成本,"54,360","1,561.3",348.1,"35,246","1,276.5",276.1,NaN,NaN,NaN
3,原材料、燃料及动力,"41,686","1,561.3",267.0,"24,827","1,276.5",194.5,"28,739","1,440.4",199.5
4,人工成本,"3,277","1,561.3",21.0,"2,307","1,276.5",18.1,"2,371","1,440.4",16.5
5,维修费,"1,808","1,561.3",11.6,"2,086","1,276.5",16.3,NaN,NaN,NaN
6,折旧及摊销,"5,254","1,561.3",33.7,"5,056","1,276.5",39.6,"5,481","1,440.4",38.1
7,其他成本,"2,315","1,561.3",14.8,970,"1,276.5",7.6,"1,238","1,440.4",8.5
8,其他业务成本,"4,833","1,561.3",NaN,"2,852","1,277.5",NaN,52,"1,440.4",NaN


In [15]:
model.elec_cost

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,NaN,2022年上半年,NaN,NaN,2021年,NaN,NaN,2020年,NaN,NaN,2019年,NaN,NaN,2018年,NaN,NaN,2017年,NaN,NaN
1,NaN,成本\n(百万元),煤电售电量\n(亿千瓦时),单位成本\n(元/兆瓦时),成本\n(百万元),煤电售电量\n(亿千瓦时),单位成本\n(元/兆瓦时),成本\n(百万元),煤电售电量\n(亿千瓦时),单位成本\n(元/兆瓦时),成本\n(百万元),煤电售电量\n(亿千瓦时),单位成本\n(元/兆瓦时),成本\n(百万元),煤电售电量\n(亿千瓦时),单位成本\n(元/兆瓦时),成本\n(百万元),煤电售电量\n(亿千瓦时),单位成本\n(元/兆瓦时)
2,原材料、燃料及动力,"22,311",796.0,280.3,"39,944","1,516.4",263.4,"23,103","1,232.7",187.4,"26,802","1,265.4",211.8,"50,511","2,675.9",188.8,"46,008","2,462.5",186.8
3,人工成本,"2,025",796.0,25.4,"3,195","1,516.4",21.1,"2,265","1,232.7",18.4,"2,328","1,265.4",18.4,"4,453","2,675.9",16.6,"3,939","2,462.5",16.0
4,维修费,860,796.0,10.8,"1,688","1,516.4",11.1,"1,989","1,232.7",16.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,折旧及摊销,"2,969",796.0,37.3,"5,078","1,516.4",33.5,"4,886","1,232.7",39.6,"5,294","1,265.4",41.8,"8,005","2,675.9",29.9,"9,235","2,462.5",37.5
6,其他成本,"1,001",796.0,12.6,"2,161","1,516.4",14.3,841,"1,232.7",6.8,"1,125","1,265.4",8.9,"1,801","2,675.9",6.7,"1,925","2,462.5",7.8


In [16]:
def preprocessing(df, cost):
    df = df.copy()
    df.loc[0] = df.loc[0].fillna(method='ffill').fillna('')
    df.columns = df.loc[0]
    # power_sale = power_sale.drop(columns=['Remove'])
    df.iloc[1] = df.iloc[1].str.replace('\n', ' ')
    columns = df.iloc[1] + ' ' + df.iloc[0]
    if cost:
        columns = columns.fillna('成本类别')
    df.columns = columns
    df.drop(index=[0, 1], inplace=True)
    df.reset_index(drop=True, inplace=True)
    # replace other symbol of None value to None
    df.replace(['-', '/', '  -   ', ' – '], None, inplace=True)

    return df

In [17]:
preprocessing(df = model.elec_cost, cost = True)

,成本类别,成本 (百万元) 2022年上半年,煤电售电量 (亿千瓦时) 2022年上半年,单位成本 (元/兆瓦时) 2022年上半年,成本 (百万元) 2021年,煤电售电量 (亿千瓦时) 2021年,单位成本 (元/兆瓦时) 2021年,成本 (百万元) 2020年,煤电售电量 (亿千瓦时) 2020年,单位成本 (元/兆瓦时) 2020年,成本 (百万元) 2019年,煤电售电量 (亿千瓦时) 2019年,单位成本 (元/兆瓦时) 2019年,成本 (百万元) 2018年,煤电售电量 (亿千瓦时) 2018年,单位成本 (元/兆瓦时) 2018年,成本 (百万元) 2017年,煤电售电量 (亿千瓦时) 2017年,单位成本 (元/兆瓦时) 2017年
0,原材料、燃料及动力,"22,311",796.0,280.3,"39,944","1,516.4",263.4,"23,103","1,232.7",187.4,"26,802","1,265.4",211.8,"50,511","2,675.9",188.8,"46,008","2,462.5",186.8
1,人工成本,"2,025",796.0,25.4,"3,195","1,516.4",21.1,"2,265","1,232.7",18.4,"2,328","1,265.4",18.4,"4,453","2,675.9",16.6,"3,939","2,462.5",16.0
2,维修费,860,796.0,10.8,"1,688","1,516.4",11.1,"1,989","1,232.7",16.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,折旧及摊销,"2,969",796.0,37.3,"5,078","1,516.4",33.5,"4,886","1,232.7",39.6,"5,294","1,265.4",41.8,"8,005","2,675.9",29.9,"9,235","2,462.5",37.5
4,其他成本,"1,001",796.0,12.6,"2,161","1,516.4",14.3,841,"1,232.7",6.8,"1,125","1,265.4",8.9,"1,801","2,675.9",6.7,"1,925","2,462.5",7.8


In [18]:
df = model.power_cost.copy()
df

,0,1,2,3,4,5,6,7,8,9
0,NaN,2021年,NaN,NaN,2020年,NaN,NaN,2019年,NaN,NaN
1,NaN,成本\n(百万元),售电量\n(亿千瓦时),单位成本\n(元/兆瓦时),成本\n(百万元),售电量\n(亿千瓦时),单位成本\n(元/兆瓦时),成本\n(百万元),售电量\n(亿千瓦时),单位成本\n(元/兆瓦时)
2,售电成本,"54,360","1,561.3",348.1,"35,246","1,276.5",276.1,NaN,NaN,NaN
3,原材料、燃料及动力,"41,686","1,561.3",267.0,"24,827","1,276.5",194.5,"28,739","1,440.4",199.5
4,人工成本,"3,277","1,561.3",21.0,"2,307","1,276.5",18.1,"2,371","1,440.4",16.5
5,维修费,"1,808","1,561.3",11.6,"2,086","1,276.5",16.3,NaN,NaN,NaN
6,折旧及摊销,"5,254","1,561.3",33.7,"5,056","1,276.5",39.6,"5,481","1,440.4",38.1
7,其他成本,"2,315","1,561.3",14.8,970,"1,276.5",7.6,"1,238","1,440.4",8.5
8,其他业务成本,"4,833","1,561.3",NaN,"2,852","1,277.5",NaN,52,"1,440.4",NaN


In [52]:
is_cost = False
df = model.power_sale.copy()
######
# column names reformatting
df.loc[0] = df.loc[0].str.extract(r'(\d{4})', expand=False).fillna(method='ffill')
df = df.fillna('')
df.iloc[1] = df.iloc[1].str.replace('\n', ' ').str.strip()
columns = df.iloc[1] + ' ' + df.iloc[0]

# if the data is a cost-related table
if is_cost:
    columns = columns.replace(' ', '成本类别')
df.columns = columns

# drop the unnecessary rows
df = df.iloc[2:].reset_index(drop=True)
df.columns = [col.strip() for col in df.columns]

# Identify the columns containing a year in the first row
year_columns = [col for col in df.columns if re.search(r'\d{4}$', col)]
new_column_names = {col: re.sub(r'\s*\d{4}\s*$', '', col) for col in df.columns}
# Preprocessing translate to the time-series panel data
for col in year_columns:
    match = re.search(r'\d{4}', col)
    year = match.group(0)
    df[col] = df[col].apply(lambda x: str(x) + '!' + str(year))
    # If Cost Category
    if is_cost:
        df[col] = df['成本类别'] + '!' + df[col]
    # if power sale
    else:
        df[col] = df['电厂分类'] + '!' + df['电厂'] + '!' + df['所在电网'] + '!' + df['地理位置'] + '!' + df[col]

# Drop and will be adding back afterward
if is_cost:
    df.drop(['成本类别'], axis=1, inplace=True)
else:
    df.drop(['电厂分类','电厂','所在电网','地理位置'], axis=1, inplace=True)

# Rename the cok
df.rename(columns=new_column_names, inplace=True)
# Drop the unnecessary columns
if not is_cost:
    df.drop(['预测毛利润 (百万元)','计算I38中的加权平均售电电价','计算Q38中的加权平均售电电价'], axis=1, inplace=True)
# Get unique column names
unique_columns = df.columns.unique()

# Create an empty DataFrame to store the concatenated columns
concatenated_df = pd.DataFrame()

# Loop through unique column names and concatenate the columns vertically
for unique_col in unique_columns:
    # Create an empty list to store values
    values = []

    # Loop through columns with the same name
    for col in df.columns[df.columns == unique_col]:
        # Extend the values list with non-NaN values from the column
        values.extend(df[col].values.tolist())

    # Flatten the nested list
    if len(values) != 1:
        values = [item for sublist in values for item in sublist]

    # Add the values list as a column in the concatenated DataFrame
    concatenated_df[unique_col] = pd.Series(values)


# Replace the original DataFrame with the concatenated one
df = concatenated_df

# Formal step of translating to the time-series panel data
for unique_col in unique_columns:
    if is_cost:
        df[['成本类别', unique_col, '年份']] = df[unique_col].str.split('!', expand=True)
    else:
        df[['电厂分类','电厂','所在电网','地理位置', unique_col, '年份']] = df[unique_col].str.split('!', expand=True)

# let cost category as the first column
if is_cost:
    columns_to_move = '成本类别'
    cols = df.columns.tolist()
    cols.insert(0, cols.pop(cols.index(columns_to_move)))
    df = df[cols].sort_values(by=['年份']).reset_index(drop=True)
# power sale
else:
    columns_to_move = ['电厂分类', '电厂', '所在电网', '地理位置']
    # Reorder columns by moving the specified columns to the front
    new_columns = columns_to_move + [col for col in df.columns if col not in columns_to_move]
    # Update the DataFrame with the new column order
    df = df[new_columns].sort_values(by=['年份']).reset_index(drop=True)

# Empty Values processing
df.replace(['-', '/', '  -   ', ' – ', ''], np.nan, inplace=True)

In [35]:
df.columns, unique_columns

(Index(['总发电量 (亿千瓦时)', '总售电量 (亿千瓦时)', '平均利用小时 (小时)', '售电标准煤耗 (克/千瓦时)',
        '售电电价 (元/兆瓦时)', '年底总装机容量 (兆瓦)', '年底权益装机容量 (兆瓦)', '计算I38中的加权平均售电电价',
        '预测毛利润 (百万元)', '计算Q38中的加权平均售电电价'],
       dtype='object'),
 Index(['总发电量 (亿千瓦时)', '总售电量 (亿千瓦时)', '平均利用小时 (小时)', '售电标准煤耗 (克/千瓦时)',
        '售电电价 (元/兆瓦时)', '年底总装机容量 (兆瓦)', '年底权益装机容量 (兆瓦)', '计算I38中的加权平均售电电价',
        '预测毛利润 (百万元)', '计算Q38中的加权平均售电电价'],
       dtype='object'))

In [53]:
df

,电厂分类,电厂,所在电网,地理位置,总发电量 (亿千瓦时),总售电量 (亿千瓦时),平均利用小时 (小时),售电标准煤耗 (克/千瓦时),售电电价 (元/兆瓦时),年底总装机容量 (兆瓦),年底权益装机容量 (兆瓦),年份
0,风能,珠海风能,南方电网,广东,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
1,燃煤,宁东电力,西北电网,宁夏,115,110,"5,705",280,339,"2,020","1,212",2018
2,燃煤,盘山电力,华北电网,天津,69,65,"5,202",302,226,"1,320",749,2018
3,燃煤,九江电力,华中电网,江西,28,25,"4,207",363,234,660,660,2018
4,燃煤,四川能源,四川电网,四川,51,48,"4,807",312,355,"1,060",482,2018
...,...,...,...,...,...,...,...,...,...,...,...,...
539,燃煤,胜利能源,华北电网,内蒙古,9.4,8.4,712.0,352.0,278.0,"1,320.0","1,320.0",2021
540,燃煤,孟津电力,华中电网,河南,48.8,45.6,"4,068.0",303.0,295.0,"1,200.0",612.0,2021
541,燃煤,四川能源,四川电网,四川,61.0,56.1,"3,468.0",325.0,424.0,"3,260.0","2,102.0",2021
542,燃煤,神木电力,西北电网,陕西,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021


In [ ]:
with open('local_dictionary.json', "r") as f:
    translations_all = json.load(f)

In [ ]:
translations_all

In [ ]:
translate_back = {value: key for key, value in translations_all.items()}

In [ ]:
translate_back['Total electricity sales (100 million kWh) 2019']